<a href="https://colab.research.google.com/github/tomik062/AI_Project/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fetch from github

In [64]:
import requests
import os

repo_owner = 'tomik062'
repo_name = 'AI_Project'
directory_path = 'data'
url = f'https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{directory_path}'

response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes
files_in_directory = response.json()

# Create a directory to save the downloaded files
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

for item in files_in_directory:
    if item['type'] == 'file':
        download_url = item['download_url']
        file_content = requests.get(download_url).content
        file_name = os.path.join(directory_path, item['name'])

        with open(file_name, 'wb') as f:
            f.write(file_content)

        print(f"Downloaded {item['name']} to {file_name}")

Downloaded GNI_PPP_const_2021_dollars.csv to data/GNI_PPP_const_2021_dollars.csv
Downloaded Religious-Composition-percentages.csv to data/Religious-Composition-percentages.csv
Downloaded abortion-rates-by-country-2025.csv to data/abortion-rates-by-country-2025.csv
Downloaded age-at-first-marriage-by-country-2025.csv to data/age-at-first-marriage-by-country-2025.csv
Downloaded average-workweek-by-country-2025.csv to data/average-workweek-by-country-2025.csv
Downloaded female-labor-force-participation-rates-slopes.csv to data/female-labor-force-participation-rates-slopes.csv
Downloaded gender_inequality_index to data/gender_inequality_index
Downloaded happiness-cantril-ladder.csv to data/happiness-cantril-ladder.csv
Downloaded healthcare-expenditure-per-capita-ppp.csv to data/healthcare-expenditure-per-capita-ppp.csv
Downloaded high_education_female.xlsx to data/high_education_female.xlsx
Downloaded high_education_male.xlsx to data/high_education_male.xlsx
Downloaded human_development_in

extract data from data sets

In [65]:
import pandas as pd
import io
def extract_WorldBank():
    df_list=[]
    df_list.append(extract_WorldBank_file('data/healthcare-expenditure-per-capita-ppp.csv'))
    df_list.append(extract_WorldBank_file('data/physicians-per-capita.csv'))
    df_list.append(extract_WorldBank_file('data/GNI_PPP_const_2021_dollars.csv'))
    return df_list

def extract_WorldBank_file(file):
    header_row_index = None
    with open(file, 'r', encoding='utf-8') as f:
      for i, line in enumerate(f):
          if 'Country Name' in line and 'Country Code' in line:
              header_row_index = i
              break
    if(header_row_index==None):
      print("Error: Header row not found in the file.")
      return None
    df = pd.read_csv(file, delimiter=',', engine='python', skiprows=header_row_index)
    df = df.drop(columns=['Indicator Name', 'Indicator Code', '2024', 'Unnamed: 69'],errors='ignore')
    return df

In [66]:
import pandas as pd

def extract_OurWorldInData():
    df_list=[]
    df_list.append(extract_OurWorldInData_file('data/urban-population-share.csv'))
    df_list.append(extract_OurWorldInData_file('data/period-average-age-of-mothers.csv'))
    df_list.append(extract_OurWorldInData_file('data/happiness-cantril-ladder.csv'))
    df_list.append(extract_OurWorldInData_file('data/female-labor-force-participation-rates-slopes.csv'))
    return df_list
def extract_OurWorldInData_file(file_name):
    df = pd.read_csv(file_name, delimiter=',', engine='python')
    if 'Entity' not in df.columns:
        print("Error: 'Entity' column not found. Cannot filter regions.")
        return df

    country_name_mapping = {
        'Egypt': 'Egypt, Arab Rep.',
        'Bahamas': 'Bahamas, The',
        'Brunei': 'Brunei Darussalam',
        'Democratic Republic of Congo': 'Congo, Dem. Rep.',
        'Congo': 'Congo, Rep.',
        'Cape Verde': 'Cabo Verde',
        'Gambia': 'Gambia, The',
        'Iran': 'Iran, Islamic Rep.',
        'Kyrgyzstan': 'Kyrgyz Republic',
        'Saint Kitts and Nevis': 'St. Kitts and Nevis',
        'South Korea': 'Korea, Rep.',
        'Laos': 'Lao PDR',
        'Saint Lucia': 'St. Lucia',
        'North Macedonia': 'North Macedonia',
        'Micronesia (country)': 'Micronesia, Fed. Sts.',
        'Russia': 'Russian Federation',
        'Slovakia': 'Slovak Republic',
        'Syria': 'Syrian Arab Republic',
        'East Timor': 'Timor-Leste',
        'Turkey': 'Turkiye',
        'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
        'Venezuela': 'Venezuela, RB',
        'Vietnam': 'Viet Nam',
        'Yemen': 'Yemen, Rep.',
        'Hong Kong': 'Hong Kong SAR, China',
        'Macao': 'Macao SAR, China',
        'St. Martin (French part)': 'St. Martin (French part)',
        'Puerto Rico': 'Puerto Rico (US)',
        "North Korea": "Korea, Dem. People's Rep.",
        'Palestine': 'West Bank and Gaza',
        'Kosovo': 'Kosovo',
        'United States Virgin Islands': 'Virgin Islands (U.S.)',
    }

    df['Entity'] = df['Entity'].apply(lambda x: country_name_mapping.get(x, x))


    # Reshape the DataFrame from long to wide format.
    # 'Entity' and 'Code' will be the id_vars, 'Year' will be the pivot column,
    # and the remaining column (the urbanization share) will be the value.
    # We need to find the name of the urbanization share column dynamically.
    value_vars = [col for col in df.columns if col not in ['Entity', 'Code', 'Year']]

    if not value_vars:
        print("Error: Could not find the urbanization share value column.")
        return df # Return without reshaping if value column is missing

    # Assuming there is only one value column after excluding id_vars and Year
    value_name = value_vars[0]


    # Pivot the DataFrame to wide format. Missing combinations of index and columns will result in NaN.
    df_wide = df.pivot(index=['Entity', 'Code'], columns='Year', values=value_name).reset_index()

    # Rename columns to match the fertility DataFrame structure
    df_wide = df_wide.rename(columns={'Entity': 'Country Name', 'Code': 'Country Code'})

    return df_wide

In [77]:
import pandas as pd

def extract_PewReaserch():
    df_list = []
    file_path = 'data/Religious-Composition-percentages.csv'
    df = pd.read_csv(file_path)

    # Map country names to match the fertility rate dataframe
    country_name_mapping = {
        'Taiwan': 'Taiwan, China',
        'Vietnam': 'Viet Nam',
        'Russia': 'Russian Federation',
        'Iran': 'Iran, Islamic Rep.',
        'Syria': 'Syrian Arab Republic',
        'South Korea': 'Korea, Rep.',
        'North Korea': "Korea, Dem. People's Rep.",
        'Egypt': 'Egypt, Arab Rep.',
        'Yemen': 'Yemen, Rep.',
        'Slovakia': 'Slovak Republic',
        'Brunei': 'Brunei Darussalam',
        'Ivory Coast': "Cote d'Ivoire",
        'Kyrgyzstan': 'Kyrgyz Republic',
        'Laos': 'Lao PDR',
        'Micronesia, Federated States of': 'Micronesia, Fed. Sts.',
        'Saint Lucia': 'St. Lucia',
        'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
        'Venezuela': 'Venezuela, RB',
        'Hong Kong': 'Hong Kong SAR, China',
        'Macao': 'Macao SAR, China',
        'St. Martin (French part)': 'St. Martin (French part)',
        'Puerto Rico': 'Puerto Rico (US)',
        'Palestinian territories': 'West Bank and Gaza',
        'U.S. Virgin Islands': 'Virgin Islands (U.S.)',
        'Turkey': 'Turkiye',
        'Bahamas': 'Bahamas, The',
        'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
        'Republic of the Congo': 'Congo, Rep.',
        'Cape Verde': 'Cabo Verde',
        'Gambia': 'Gambia, The',
        'Czech Republic': 'Czechia',
        'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
        'Federated States of Micronesia': 'Micronesia, Fed. Sts.'

    }

    df['Country'] = df['Country'].replace(country_name_mapping)

    # Reshape the region data to have years as columns
    df_region = df.pivot_table(index=['Country', 'Countrycode'], columns='Year', values='Region', aggfunc='first').reset_index()
    df_region = df_region.rename(columns={'Country': 'Country Name', 'Countrycode': 'Country Code'})
    df_list.append(df_region)

    # Identify religion columns (between 'Population' and 'Level')
    columns = df.columns.tolist()
    try:
        population_index = columns.index('Population')
        level_index = columns.index('Level')
        religion_columns = columns[population_index + 1:level_index]
    except ValueError:
        print("Error: 'Population' or 'Level' column not found.")
        return df_list # Return with just the region dataframe if columns are missing


    # Create a dataframe for each religion
    for religion in religion_columns:
        # Select relevant columns for the current religion
        religion_df = df[['Country', 'Countrycode', 'Year', religion]].copy()

        # Pivot the DataFrame to get years as columns
        df_religion_pivot = religion_df.pivot_table(index=['Country', 'Countrycode'], columns='Year', values=religion).reset_index()
        df_religion_pivot = df_religion_pivot.rename(columns={'Country': 'Country Name', 'Countrycode': 'Country Code'})

        # Get all year columns dynamically
        year_columns = [col for col in df_religion_pivot.columns if isinstance(col, int)]
        df_religion_filtered = df_religion_pivot[['Country Name', 'Country Code'] + year_columns]


        df_list.append(df_religion_filtered)

    return df_list

In [4]:
def merge(data, df_list):
    for new_df in df_list:
      new_df_year_columns = [col for col in new_df.columns if isinstance(col, int)]
      new_df_country_map = new_df.set_index('Country Name')
      for key, value in data.items():
          country_name = key[0]
          year = key[1]
          data_list = value[1]
          if country_name in new_df_country_map.index:
              new_df_row = new_df_country_map.loc[country_name]
              if year in new_df_row.index and pd.notna(new_df_row[year]):
                      new_value = new_df_row[year]
                      data_list.append(new_value)
              #spaghetti code incoming, hope doest break anything
              elif str(year) in new_df_row.index and pd.notna(new_df_row[str(year)]):
                      new_value = new_df_row[str(year)]
                      data_list.append(new_value)
              else:
                      data_list.append(float('nan'))

          else:
              print(f"Warning: Country '{country_name}' not found in the new DataFrame.")
              data_list.append(float('nan'))


init data for single year

In [78]:
data={}
df_fertility=extract_WorldBank_file('data/total-fertility-rate.csv')
regions_to_exclude_by_code = [
    'CEB', 'EAR', 'IBD', 'LTE', 'MAF', 'PSS', 'AFE', 'AFW', 'ARB', 'CSS', 'EAP', 'EAS', 'ECA', 'ECS', 'EMU', 'EUU', 'FCS', 'HIC', 'HPC',
    'IDA', 'IDB', 'IDX', 'LAC', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY', 'MEA', 'MIC', 'MNA',
    'NAC', 'OED', 'OSS', 'PRE', 'PST', 'SAS', 'SSA', 'SSF', 'SST', 'UMC', 'WLD','IBT','TEA','TEC','TLA','TSS','TMN','TSA'
]
df_fertility = df_fertility[~df_fertility['Country Code'].isin(regions_to_exclude_by_code)]
df_fertility_2023 = df_fertility[['Country Name', '2020']].copy()
df_fertility_2023['2020'] = pd.to_numeric(df_fertility_2023['2020'], errors='coerce')
df_fertility_2023.dropna(subset=['2020'], inplace=True)
for index, row in df_fertility_2023.iterrows():
    country_name = row['Country Name']
    fertility_rate = row['2020']
    data[(country_name,2020)] = (fertility_rate, [])
merge(data,extract_OurWorldInData()+extract_WorldBank()+extract_PewReaserch())
print(data)

{('Aruba', 2020): (1.662, [np.float64(48.78), np.float64(28.641), nan, nan, nan, nan, np.float64(28180.0), 'Latin America-Caribbean', np.float64(81.42006683), np.float64(0.33776018), np.float64(5.873986721), np.float64(0.544169188), np.float64(0.281466812), np.float64(0.354266971), np.float64(11.18828392)]), ('Afghanistan', 2020): (5.145, [np.float64(27.575386), np.float64(30.275), np.float64(2.523), np.float64(16.496), np.float64(324.667104880284), np.float64(0.254), np.float64(2580.0), 'Asia-Pacific', np.float64(0.019379199), np.float64(99.86196899), np.float64(0.008456565), np.float64(0.020000853), np.float64(0.000128296), np.float64(2.56593e-05), np.float64(0.090042546)]), ('Angola', 2020): (5.371, [np.float64(65.45493), np.float64(28.724), nan, nan, np.float64(217.194973649642), nan, np.float64(5870.0), 'Sub-Saharan Africa', np.float64(93.04470825), np.float64(0.255656004), np.float64(6.114808083), np.float64(0.002393168), np.float64(0.009273528), np.float64(0.001196584), np.float